In [1]:
import numpy as np
import glob
import pickle


In [17]:
path_to_save_mri_data = '/usr/local/micapollo01/MIC/DATA/STUDENTS/mvhave7/Results/Preprocessing/mri/'
path_to_save_grappa_data = '/usr/local/micapollo01/MIC/DATA/STUDENTS/mvhave7/Results/Preprocessing/grappa/'


file_paths_train = sorted(glob.glob(path_to_save_mri_data+"training_data_GrappaNet_16_coils_batch_*.npy"))[50]
file_paths_train_GT = sorted(glob.glob(path_to_save_mri_data+"training_data_GT_GrappaNet_16_colis_batch_*.npy"))[50]
file_paths_val = sorted(glob.glob(path_to_save_mri_data+"validation_data_GrappaNet_16_coils_batch_*.npy"))[50]
file_paths_val_GT = sorted(glob.glob(path_to_save_mri_data+"validation_data_GT_GrappaNet_16_colis_batch_*.npy"))[50]
file_paths_grappa_indx_train = sorted(glob.glob(path_to_save_grappa_data+"grappa_train_indx_GrappaNet_16_coils_batch_*.npy"))[50]
file_paths_grappa_indx_val = sorted(glob.glob(path_to_save_grappa_data+"grappa_test_indx_GrappaNet_16_colis_batch_*.npy"))[50]
file_paths_grappa_wt = sorted(glob.glob(path_to_save_grappa_data+"grappa_wt_batch_*.pickle"))[50]
file_paths_grappa_p = sorted(glob.glob(path_to_save_grappa_data+"grappa_p_batch_*.pickle"))[50]


In [18]:
x_train = np.load(file_paths_train)
y_train = np.load(file_paths_train_GT)
grappa_train_indx = np.load(file_paths_grappa_indx_train)
with open(file_paths_grappa_wt, 'rb') as handle:
    grappa_wt = pickle.load(handle)
with open(file_paths_grappa_p, 'rb') as handle:
    grappa_p = pickle.load(handle)
x_test = np.load(file_paths_val)
y_test = np.load(file_paths_val_GT)
grappa_test_indx = np.load(file_paths_grappa_indx_val)
with open(file_paths_grappa_wt, 'rb') as handle:
    grappa_wt = pickle.load(handle)
with open(file_paths_grappa_p, 'rb') as handle:
    grappa_p = pickle.load(handle)


In [19]:
print(x_train.dtype)
print(y_train.dtype)
print(grappa_train_indx.dtype)
print(grappa_test_indx.dtype)
print(x_test.dtype)
print(y_test.dtype)


float32
float32
int64
int64
float32
float32


In [20]:
print(x_train.shape)
print(y_train.shape)
print(grappa_train_indx.shape)
print(grappa_test_indx.shape)
print(x_test.shape)
print(y_test.shape)


(138, 640, 320, 32)
(138, 640, 320)
(138,)
(16,)
(16, 640, 320, 32)
(16, 640, 320)


In [21]:
print(type(x_train))
print(type(y_train))
print(type(grappa_train_indx))
print(type(grappa_test_indx))
print(type(x_test))
print(type(y_test))
print(type(grappa_wt))
print(type(grappa_p))


<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'list'>
<class 'list'>


Compare the above data properties with the data properties of the variables when they were originally preprocessed

In [1]:
import h5py, os
import numpy as np
from numpy import fft 
import matplotlib.pyplot as plt
from utils import estimate_mdgrappa_kernel, calculate_mask, comp_sub_kspace, comp_img
from pathlib import Path
from itertools import chain
import gc
import time
import pickle

clustered_data_2 = np.load("/usr/local/micapollo01/MIC/DATA/STUDENTS/mvhave7/Results/Preprocessing/exploration/16coil_slice_size_clustered_fastmri_data.npy", allow_pickle=True)
clustered_data_2 = clustered_data_2.item()

files_16_640_320 = clustered_data_2[(640,320)]
training_files = sorted(files_16_640_320)

crop_size = (32,640,320)


In [2]:
## Pick a small batch to view the data properties after the original preprocessing process is done

remaining_files = training_files[0:2]


## Create training data pairs and estimate GRAPPA kernels

print('Starting dataset construction and GRAPPA kernel estimation...')

cnt = 1
last_mask = None
X_train = []
Y_train = []
grappa_wt = []
grappa_p = []

for mri_f in remaining_files:
    filename = os.path.basename(mri_f)
    filename = filename.replace(".h5","")
    with h5py.File(mri_f,'r') as f:

        k = f['kspace'][()]
        sequence = f.attrs['acquisition']
        nSL, nCh, nFE, nPE = k.shape
            
        # Create the subsampled training data. In the GrappaNet paper, they say they performed experiments for R=4 and R=8, but they never mention the number or fraction of ACS lines used...
        mask = calculate_mask(nFE,nPE,0.08,4)
        last_mask = mask
        subsampled_k = k*mask
        
        for slice in range(nSL): 
            target_img = np.zeros((nCh,nFE,nPE),dtype=np.float)
            sub_kspace = np.zeros((nCh*2,nFE,nPE),dtype=np.float)
            wt,ps = estimate_mdgrappa_kernel(kspace=subsampled_k[slice,:,:,:],calib=None,kernel_size=(5,5),coil_axis=0) 
            for iCh in range(nCh):
                    target_img[iCh,:,:] = abs(fft.fftshift(fft.ifft2(k[slice,iCh,:,:])))
                    sub_kspace[iCh,:,:] = subsampled_k[slice,iCh,:,:].real
                    sub_kspace[iCh+nCh,:,:] = subsampled_k[slice,iCh,:,:].imag
            X_train.append(list(comp_sub_kspace(sub_kspace,crop_size)))
            Y_train.append(list(comp_img(target_img,(crop_size[0]//2,crop_size[1],crop_size[2]))))
            grappa_wt.append(wt)
            grappa_p.append(ps)
          
        print(cnt,filename,sequence,nSL,nCh,nFE,nPE,sub_kspace.shape)
        cnt += 1

Y_train = np.array(Y_train).astype(np.float32)
X_train = np.array(X_train).astype(np.float32)


print('Done. Calculating RSS images as references for the loss function...')


## Calculate RSS images that will be used as references for the loss function

X_train = np.transpose(X_train,(0,2,3,1))
Y_rss = np.sqrt(np.sum(np.square(Y_train),axis=1))
Y_rss = Y_rss.astype(np.float32)
print(X_train.shape,Y_rss.shape)


print('Done. Normalizing the data...')


## Normalize the data

dims = X_train.shape
for i in range(dims[0]):
    for j in range(dims[3]):
        X_train[i,:,:,j] = X_train[i,:,:,j]/((np.max(X_train[i,:,:,j])-np.min(X_train[i,:,:,j]))+1e-10)

for i in range(dims[0]):
    Y_rss[i,:,:] = Y_rss[i,:,:]/((np.max(Y_rss[i,:,:])-np.min(Y_rss[i,:,:]))+1e-10)


print('Done. Performing a datasplit...')


## Create a dataset split 90-10 training-validation

x_train = X_train[0:int(X_train.shape[0]-X_train.shape[0]*0.1),:,:,:]
y_train = Y_rss[0:int(X_train.shape[0]-X_train.shape[0]*0.1),:,:]
x_test = X_train[int(X_train.shape[0]-X_train.shape[0]*0.1):,:,:,:]
y_test = Y_rss[int(X_train.shape[0]-X_train.shape[0]*0.1):,:,:]
y_test = np.reshape(y_test, (y_test.shape[0],crop_size[1],crop_size[2]))
grappa_train_indx = np.array(range(0,int(X_train.shape[0]-X_train.shape[0]*0.1)),dtype=int)
grappa_test_indx = np.array(range(int(X_train.shape[0]-X_train.shape[0]*0.1),X_train.shape[0]),dtype=int)


print('Done. Checking data specs of the results...')


print(x_train.dtype)
print(y_train.dtype)
print(grappa_train_indx.dtype)
print(grappa_test_indx.dtype)
print(x_test.dtype)
print(y_test.dtype)

print(x_train.shape)
print(y_train.shape)
print(grappa_train_indx.shape)
print(grappa_test_indx.shape)
print(x_test.shape)
print(y_test.shape)

print(type(x_train))
print(type(y_train))
print(type(grappa_train_indx))
print(type(grappa_test_indx))
print(type(x_test))
print(type(y_test))
print(type(grappa_wt))
print(type(grappa_p))


print('Done.')


Starting dataset construction and GRAPPA kernel estimation...


/home/mvhave7/GitLab/master_thesis/GrappaNet/model_v2/utils.py:97: RuntimeWarning: invalid value encountered in double_scalars
  np.array(pads, dtype=np.uintp), lamda)


1 file_brain_AXFLAIR_200_6002435 AXFLAIR 16 16 640 320 (32, 640, 320)
2 file_brain_AXFLAIR_200_6002442 AXFLAIR 16 16 640 320 (32, 640, 320)
Done. Calculating RSS images as references for the loss function...
(32, 640, 320, 32) (32, 640, 320)
Done. Normalizing the data...
Done. Performing a datasplit...
Done. Checking data specs of the results...
float32
float32
int64
int64
float32
float32
(28, 640, 320, 32)
(28, 640, 320)
(28,)
(4,)
(4, 640, 320, 32)
(4, 640, 320)
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'list'>
<class 'list'>
Done.
